In [1]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc
import os
import csv
import itertools
import tensorflow.contrib.slim as slim
%matplotlib inline

In [2]:
def read_img(file_names):
    #file_names = './data-moveImg/moveImg-0.bmp'
    image0 = scipy.misc.imread( file_names )

    img_size = 400

    tmp_img = []
    for i1 in xrange(img_size):
        tmp_img.append( [] )
        for i2 in xrange(img_size):
            if image0[i1][i2][0] == 0:
                tmp_img[i1].append( [0] )
            else:
                tmp_img[i1].append( [1] )
                
    image0 = np.array( tmp_img )
    image0=image0.reshape(img_size*img_size)
    return image0

In [4]:
def batch_obtain(dir_name):
    # dir_name = 'data-moveImg-39x39-test'
    import re
    data_keep = []

    ink_code = open( './'+dir_name+'/ink-code.txt' )
    for lines in ink_code:
        # print lines
        searchObj = re.search( r'create\_picture\((\d+)\, (\d+)\, \"moveImg\-(\d+)\.bmp\"\)\;', lines, re.M|re.I)
        if searchObj: 
            tmp=[]
            for i in range(3):
                tmp.append( int( searchObj.group(i+1) ) )
            data_keep.append( tmp )

    data_keep=np.array( data_keep )

    x_batch = []
    y_batch = []

    for i in xrange( data_keep.shape[0] ): # data_keep.shape[0]
        image = read_img( './'+dir_name+'/moveImg-'+str( data_keep[i][2] )+'.bmp' )
        x_batch.append( image )
        y_batch.append( [data_keep[i][0], data_keep[i][1]] )

    y_batch = np.array( y_batch )

    return x_batch, y_batch

In [5]:
x_batch, y_batch = batch_obtain( 'data-moveImg' )

In [6]:
x_test, y_test = batch_obtain( 'data-moveImg-test' )

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)

### Implementing the network itself

In [7]:
class network():
    def __init__(self):
        
        myScope = 'main'
        
        self.x_input =  tf.placeholder(shape=[None,400*400],dtype=tf.float32)
        imageIn = tf.reshape(self.x_input,shape=[-1,400, 400,1])

        self.conv1 = slim.convolution2d( \
            inputs=imageIn,num_outputs=10,\
            kernel_size=[5,5],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv1')
        
        # tf.nn.relu
        
        self.pool1 = self.max_pool_2x2( self.conv1 )
        
        self.conv2 = slim.convolution2d( \
            inputs=self.pool1,num_outputs=20,\
            kernel_size=[4,4],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv2')
        
        self.pool2 = self.max_pool_2x2( self.conv2 )
        
        self.conv3 = slim.convolution2d( \
            inputs=self.pool2,num_outputs=30,\
            kernel_size=[4,4],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv3')
        
        self.pool3 = self.max_pool_2x2( self.conv3 )
        
        self.conv4 = slim.convolution2d( \
            inputs=self.pool3,num_outputs=40,\
            kernel_size=[4,4],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv4')

        self.pool4 = self.max_pool_2x2( self.conv4 )
        
        self.conv5 = slim.convolution2d( \
            inputs=self.pool4,num_outputs=50,\
            kernel_size=[3,3],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv5')

        self.pool5 = self.max_pool_2x2( self.conv5 )
        
        self.conv6 = slim.convolution2d( \
            inputs=self.pool5,num_outputs=60,\
            kernel_size=[3,3],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv6')

        self.pool6 = self.max_pool_2x2( self.conv6 )
        
        self.conv7 = slim.convolution2d( \
            inputs=self.pool6,num_outputs=70,\
            kernel_size=[3,3],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv7')     

        self.pool7 = self.max_pool_2x2( self.conv7 )
        
        self.conv8 = slim.convolution2d( \
            inputs=self.pool7,num_outputs=80,\
            kernel_size=[2,2],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv8')     

        #print self.conv8.shape # 1,1,80

        self.conv8 = tf.reshape(self.conv8, shape=[-1,80])

        x_dim = 80
        y_dim = 2
        # network weights
        hid_dim = 120
        W1 = weight_variable([x_dim,hid_dim])
        b1 = bias_variable([hid_dim])
        W2 = weight_variable([hid_dim,y_dim])
        b2 = bias_variable([y_dim])

        # hidden layers
        h_layer = tf.nn.relu(tf.matmul(self.conv8,W1) + b1)
        # Q Value layer
        self.y_value = tf.matmul(h_layer,W2) + b2


        self.y_input = tf.placeholder("float",[None,y_dim])

        self.global_step = tf.Variable(0, name="global_step", trainable=False)
    
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    
        self.loss = tf.reduce_mean(tf.square(self.y_input - self.y_value))
        self.trainable_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="q_network")
        self.gradients = self.optimizer.compute_gradients(self.loss)
        self.train_op = self.optimizer.apply_gradients(self.gradients, global_step=self.global_step)
    def max_pool_2x2(self, x):
        return tf.nn.max_pool( x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME' )

In [20]:
load_model = True #Whether to load a saved model.

In [8]:
load_model = False #Whether to load a saved model.

### Training the network

In [9]:
from tqdm import trange

In [10]:
model_path = './INK7862-01-400x400'

In [11]:
tf.reset_default_graph()

mainNN = network() 

init = tf.global_variables_initializer() 

saver = tf.train.Saver()

#Make a path for our model to be saved in.
if not os.path.exists( model_path ):
    os.makedirs( model_path )

with tf.Session() as sess:
    
    sess.run(init)
    
    if load_model == True:
        load_path = saver.restore(sess, model_path+'/model.ckpt')
    
    for _ in trange(100):
        sess.run([mainNN.train_op], {mainNN.y_input:y_batch, mainNN.x_input:x_batch})

    A = mainNN.y_value.eval(feed_dict = {
      mainNN.x_input:x_batch
      })
    print A

    save_path = saver.save(sess, model_path+'/model.ckpt')

100%|██████████| 100/100 [8:53:09<00:00, 1550.44s/it]  


[[ 199.18890381  156.06910706]
 [ 228.34843445  287.46594238]
 [ 244.59992981  291.1267395 ]
 [ 227.81515503  226.9959259 ]
 [ 244.73129272  196.65002441]
 [ 122.33783722  141.46949768]
 [ 201.67950439  270.41479492]
 [ 126.29177856  156.76597595]
 [ 263.07171631  156.79115295]
 [ 172.51237488  105.99022675]
 [ 185.02536011  121.83995056]
 [ 185.88702393  228.58570862]
 [ 247.14610291  167.18537903]
 [ 243.90228271  121.96885681]
 [ 277.9654541   267.37905884]
 [ 232.30065918  285.50546265]
 [ 206.2739563   233.78373718]
 [ 170.22796631  237.20773315]
 [ 145.3996582   237.49960327]
 [ 142.48924255  161.35472107]
 [ 224.99383545  283.37692261]
 [ 215.21943665  133.91117859]
 [ 217.412323    128.12364197]
 [ 143.09088135  266.60412598]
 [ 191.09112549  106.561203  ]
 [ 216.48474121  118.76688385]
 [ 168.30407715  232.74601746]
 [ 174.0737915   113.07722473]
 [ 165.56504822  126.64661407]
 [ 105.14173889  199.24430847]
 [ 233.40275574  283.8069458 ]
 [ 119.54187775  204.26641846]
 [ 280.7

In [12]:
print y_batch[:10]

[[197 165]
 [229 282]
 [248 294]
 [233 227]
 [246 199]
 [116 131]
 [196 264]
 [120 148]
 [257 165]
 [165 105]]


In [23]:
tf.reset_default_graph()

mainNN = network() 

init = tf.global_variables_initializer() 

saver = tf.train.Saver()

#Make a path for our model to be saved in.
if not os.path.exists( model_path ):
    os.makedirs( model_path )

with tf.Session() as sess:
    
    sess.run(init)
    
    if load_model == True:
        load_path = saver.restore(sess, model_path+'/model.ckpt')
        
    A = mainNN.y_value.eval(feed_dict = {
      mainNN.x_input:x_test
      })
    print A

[[ 211.72872925  115.20989227]
 [ 274.32025146  277.84960938]
 [ 198.51042175  185.9443512 ]
 [ 133.18554688  209.63446045]
 [ 181.89743042  259.66748047]
 [ 161.39906311  290.68426514]
 [ 157.31033325  249.30621338]
 [ 270.05783081  250.41920471]
 [ 142.70262146  228.50517273]
 [ 265.74124146  122.12697601]
 [ 276.8742981   265.4239502 ]
 [ 120.38916779  250.73477173]
 [ 201.66120911  221.7709198 ]
 [ 203.0934906   101.45345306]
 [ 219.14845276  272.38052368]
 [ 119.54445648  166.17044067]
 [ 102.47396088  182.84828186]
 [ 114.69072723  243.91673279]
 [ 140.34303284  125.96865845]
 [ 137.57141113  237.27082825]
 [ 103.42129517  178.93664551]
 [ 271.49478149  118.21642303]
 [ 239.69921875  285.29907227]
 [ 270.29440308  114.66938019]
 [ 153.59059143  282.94628906]
 [ 272.06185913  273.10467529]
 [ 252.69248962  274.77975464]
 [ 175.4198761   248.29905701]
 [ 191.25570679  178.55253601]
 [ 116.58660889  237.30906677]
 [ 247.84809875  284.73916626]
 [ 280.92510986  121.56560516]
 [ 202.2

In [24]:
print y_test[:5], y_test[-5:]

[[210 119]
 [281 289]
 [203 189]
 [138 203]
 [183 263]] [[125 265]
 [149 131]
 [272 129]
 [185 282]
 [145 139]]
